Pontificia Universidad Católica de Chile <br>
Departamento de Ciencia de la Computación <br>
IIC2433 - Minería de Datos
<br>

<center>
    <h2> Proyecto Minería </h2>
    <h1> Análisis causal de los factores  que  <br> impulsan el rendimiento estudiantil</h1>
    <p>
        Profesor Maximiliano Ojeda<br>
        Segundo Semestre 2025<br>    
        Fecha de entrega: 28 de octubre
    </p>
    <br>
</center>

<br>


Integrantes: Clemente Acevedo, Paul Alvarez, Lucciano Bertinelli. <br>


---

# Librerías

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import seaborn as sns

# Librerías adicionales utilizadas
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split,  cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier


# Introducción: 
El presente análisis utiliza datos del High School Longitudinal Study of 2009 (HSLS:09), un estudio realizado por el National Center for Education Statistics (NCES) de los Estados Unidos. Este dataset sigue a más de 23.000 estudiantes desde el 9° grado (año 2009) hasta su transición a la educación superior y mercado laboral. El objetivo del HSLS:09 es comprender cómo los factores académicos, familiares y contextuales influyen en las trayectorias educativas y profesionales de los estudiantes. Su diseño longitudinal permite analizar tanto asociaciones como posibles efectos causales a lo largo del tiempo.

Los datos son de uso público [High School Longitudinal Study of 2009](https://nces.ed.gov/datalab/onlinecodebook), disponibles en el portal oficial del NCES.

Para este proyecto, se descargaron los archivos de uso público en formato CSV desde el Online Codebook del NCES. Se utilizó principalmente el archivo HSLS09_STUDENT.csv, que contiene la información individual de los estudiantes, junto con el archivo HSLS09_TRANSCRIPT.csv, que incluye registros de rendimiento académico (GPA y créditos cursados).

A partir de estas fuentes se seleccionaron variables clave relacionadas con tratamiento (participación en programas académicos avanzados o de apoyo; cursos “Honors”, “Dual Credit”, o tutorías), de resultado (desempeño académico medido por el promedio de calificaciones GPA) y confusoras (nivel educativo de los padres, ingreso familiar, rendimiento previo y variables demográficas).

El uso de este dataset permite realizar un análisis causal robusto sobre los factores que influyen en el rendimiento estudiantil, combinando información socioeconómica y educativa.

# Parte 1: Carga de la base de datos y selección de variables

## 1.1 Carga de Datos y vista previa

In [23]:
# Cargamos la base de datos 
df = pd.read_csv("hsls_17_student_pets_sr_v1_0.csv")

In [24]:
# Tamaño inicial
print(f"El tamaño del dataset es de {df.shape[0]} registros y {df.shape[1]} variables.")

El tamaño del dataset es de 23503 registros y 9614 variables.


## 1.2 Selección de columnas

In [25]:
# Seleccionamos nuestras variables a utilizar
cols = [
    "STU_ID", "SCH_ID",
    "X3TGPA9TH", "X1TXMTH",
    "X1PAREDU", "X1FAMINCOME",
    "X1POVERTY", "X1POVERTY130",
    "X1PAR1EMP",
    "X1SEX", "X1RACE", "X1STDOB",
    "C1GETAHEAD", "X3TCREDADV", "X3TCREDHON", "X5DUALCRSFLG", "A1G9STUDY",
    "X3TGPAACAD", "X3TGPA12TH",
    "X5GPAALL", "X3TXSATCOMP"
]

# Nuevo dataset con las variables listas
df_vars = df[cols]

In [26]:
print(f"El tamaño del dataset es de {df_vars.shape[0]} registros y {df_vars.shape[1]} variables.")

El tamaño del dataset es de 23503 registros y 21 variables.


In [27]:
# Finalmente guardamos el dataset actual
df_vars.to_csv("hsls09_filtrado.csv", index=False)

## 1.3 Descripción de variables utilizadas


#### **1. Llaves de Unión**
- **STU_ID**: *Integer* que identifica de forma única a cada estudiante dentro del estudio.  
- **SCH_ID**: *Integer* que identifica la escuela a la que pertenece el estudiante.


#### **2. Confusores (El Contexto – Ola Base X1/X3)**

**Habilidad Previa**
- **X3TGPA9TH**: *Float* que representa el promedio general (GPA) del estudiante en 9° grado. Indica su rendimiento académico previo.  
- **X1TXMTH**: *Float* que mide la aptitud matemática inicial del estudiante mediante un puntaje estandarizado aplicado en 9° grado.

**Familia**
- **X1PAREDU**: *Categórica ordinal* que indica el nivel educativo más alto alcanzado por los padres o tutores.  
- **X1FAMINCOME**: *Float* que representa el ingreso familiar anual del hogar.  
- **X1POVERTY**: *Binaria* que indica si el ingreso familiar está por debajo del 100% del umbral de pobreza.  
- **X1POVERTY130**: *Binaria* que indica si el ingreso familiar está por debajo del 130% del umbral de pobreza.   
- **X1PAR1EMP**: *Categórica nominal* que describe el estado laboral del sustentador principal del hogar.

**Estudiante**
- **X1SEX**: *Categórica nominal* que indica el sexo biológico del estudiante (masculino o femenino).  
- **X1RACE**: *Categórica nominal* que identifica la etnia o raza del estudiante.  
- **X1STDOB**: *Date* que indica la fecha de nacimiento del estudiante, útil para calcular la edad al inicio del estudio.


#### **3. Tratamiento (La Causa)**
- **C1GETAHEAD**: *Binaria* que indica si el estudiante participó en cursos de verano o actividades diseñadas para adelantar contenidos académicos.  
- **X3TCREDADV**: *Float* que mide la cantidad total de créditos obtenidos en cursos avanzados (AP o IB).  
- **X3TCREDHON**: *Float* que indica el número de créditos obtenidos en cursos “Honors” o de alta exigencia.  
- **X5DUALCRSFLG**: *Binaria* que identifica si el estudiante cursó asignaturas “dual credit” o de formación preuniversitaria.  
- **A1G9STUDY**: *Binaria* que indica si la escuela enseña o promueve técnicas de estudio como parte del currículo académico.


#### **4. Resultado (El Efecto)**
- **X3TGPAACAD**: *Float* que representa el GPA académico general acumulado del estudiante durante la enseñanza media.  
- **X3TGPA12TH**: *Float* que refleja el promedio general (GPA) obtenido en el último año de high school (12° grado).


#### **(Futuro – Seguimiento postsecundario)**
- **X5GPAALL**: *Float* que indica el GPA promedio de todas las instituciones de educación superior cursadas por el estudiante (medido en 2017).  
- **X3TXSATCOMP**: *Float* que representa el puntaje total combinado en pruebas estandarizadas SAT/ACT, como indicador adicional de desempeño académico.


# Parte 2: Limpieza del dataset y Normalización 

In [3]:
# Cargamos la base de datos nueva
df = pd.read_csv("hsls09_filtrado.csv")
df.head()

,STU_ID,SCH_ID,X3TGPA9TH,X1TXMTH,X1PAREDU,X1FAMINCOME,X1POVERTY,X1POVERTY130,X1PAR1EMP,X1SEX,...,X1STDOB,C1GETAHEAD,X3TCREDADV,X3TCREDHON,X5DUALCRSFLG,A1G9STUDY,X3TGPAACAD,X3TGPA12TH,X5GPAALL,X3TXSATCOMP
0,10001,-5,3.5,0.8304,5,10,0,0,4,1,...,199502,-5,3.0,0.0,3,0,3.5,3.5,3.6,-5
1,10002,-5,4.0,-0.2956,3,3,0,0,3,2,...,199511,-5,0.0,0.0,3,0,4.0,4.0,2.9,-5
2,10003,-5,3.5,1.2997,7,6,0,0,4,2,...,199506,-5,6.0,4.0,3,0,2.5,3.0,2.7,-5
3,10004,-5,3.5,-0.1427,4,5,0,0,2,2,...,199505,-5,5.0,10.5,-6,0,4.0,4.0,-6.0,-5
4,10005,-5,3.5,1.1405,4,9,0,0,4,1,...,199505,-5,5.0,0.5,-6,0,3.5,3.5,-6.0,-5


## 2.1 Investigar valores nulos

# Parte 3: Línea Base Predictiva (KNN)

## 3.1 k-Nearest Neighbors